In [20]:
#Lets import the necessary libraries

import pathlib
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import PIL
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import datetime
import warnings
warnings.filterwarnings("ignore")
import abc
from sys import getsizeof
from skimage.transform import resize

from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
import keras
from imageio import imread


In [21]:
#Lets set the random seed
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
tf.random.set_seed(30)

In [22]:
# train_doc = np.random.permutation(open('/home/train.csv').readlines())
# val_doc = np.random.permutation(open('/home/val.csv').readlines())
# batch_size = 51

In [23]:
#Lets mount the drive

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [24]:
#Unzip the data

!unzip gdrive/MyDrive/Project_data

Streaming output truncated to the last 5000 lines.
 extracting: Project_data/train/WIN_20180926_17_33_08_Pro_Thumbs_Down_new/WIN_20180926_17_33_08_Pro_00029.png  
 extracting: Project_data/train/WIN_20180926_17_33_08_Pro_Thumbs_Down_new/WIN_20180926_17_33_08_Pro_00031.png  
 extracting: Project_data/train/WIN_20180926_17_33_08_Pro_Thumbs_Down_new/WIN_20180926_17_33_08_Pro_00033.png  
 extracting: Project_data/train/WIN_20180926_17_33_08_Pro_Thumbs_Down_new/WIN_20180926_17_33_08_Pro_00035.png  
 extracting: Project_data/train/WIN_20180926_17_33_08_Pro_Thumbs_Down_new/WIN_20180926_17_33_08_Pro_00037.png  
 extracting: Project_data/train/WIN_20180926_17_33_08_Pro_Thumbs_Down_new/WIN_20180926_17_33_08_Pro_00039.png  
 extracting: Project_data/train/WIN_20180926_17_33_08_Pro_Thumbs_Down_new/WIN_20180926_17_33_08_Pro_00041.png  
 extracting: Project_data/train/WIN_20180926_17_33_08_Pro_Thumbs_Down_new/WIN_20180926_17_33_08_Pro_00043.png  
 extracting: Project_data/train/WIN_20180926_17_33_08

In [26]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 10

In [27]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    temp = resize(image,(120,120))
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2]) #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = resize(image,(120,120))
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0])
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1])
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2])
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

In [28]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('Training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('Validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('Epochs =', num_epochs)

Training sequences = 663
Validation sequences = 100
Epochs = 10


In [29]:
#write your model here
#model a
model_a = Sequential()

model_a.add(Conv3D(8, #number of filters 
                 kernel_size=(3,3,3), 
                 input_shape=(30, 120, 120, 3),
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

model_a.add(Conv3D(16, #Number of filters, 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

model_a.add(Conv3D(32, #Number of filters 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

model_a.add(Conv3D(64, #Number pf filters 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_a.add(Flatten())

model_a.add(Dense(1000, activation='relu'))
model_a.add(Dropout(0.5))

model_a.add(Dense(500, activation='relu'))
model_a.add(Dropout(0.5))

#softmax layer
model_a.add(Dense(5, activation='softmax'))

In [30]:
optimiser = optimizers.Adam(lr=0.001) #write your optimizer
model_a.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_a.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_4 (Conv3D)           (None, 30, 120, 120, 8)   656       
                                                                 
 batch_normalization_4 (Batc  (None, 30, 120, 120, 8)  32        
 hNormalization)                                                 
                                                                 
 activation_4 (Activation)   (None, 30, 120, 120, 8)   0         
                                                                 
 max_pooling3d_4 (MaxPooling  (None, 15, 60, 60, 8)    0         
 3D)                                                             
                                                                 
 conv3d_5 (Conv3D)           (None, 15, 60, 60, 16)    3472      
                                                                 
 batch_normalization_5 (Batc  (None, 15, 60, 60, 16)  

In [31]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [32]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [33]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [34]:
model_a.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 4.4887 - categorical_accuracy: 0.2866 Source path =  Project_data/val ; batch size = 10

Epoch 1: saving model to model_init_2023-02-2207_54_06.822453/model-00001-4.48869-0.28658-1.34870-0.41000.h5
67/67 [==============================] - 1262s 19s/step - loss: 4.4887 - categorical_accuracy: 0.2866 - val_loss: 1.3487 - val_categorical_accuracy: 0.4100 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 2.2853 - categorical_accuracy: 0.3333
Epoch 2: saving model to model_init_2023-02-2207_54_06.822453/model-00002-2.28530-0.33333-1.39070-0.39000.h5
67/67 [==============================] - 496s 7s/step - loss: 2.2853 - categorical_accuracy: 0.3333 - val_loss: 1.3907 - val_categorical_accuracy: 0.3900 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 1.9622 - categorical_accuracy: 0.2537
Epoch 3: saving model to 

In [35]:
classes = 5 #left swipe, right swipe, thumbs up, thumbs down, stop
channel = 3
x = 30 # number of frames
y = 120 # image width
z = 120 # image height

def generator_ex(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    temp = resize(image,(y,z))
                    #Converting to gray scale
                    temp = temp.mean(axis=-1,keepdims=1) 
                    temp = temp/127.5-1 #Normalize data
                    batch_data[folder,idx] = temp #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = resize(image,(y,z))
                    #Converting to gray scale
                    temp = temp.mean(axis=-1,keepdims=1) 
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx] = temp
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

In [36]:
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

# Define model b
model_b = Sequential()
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Flatten())
model_b.add(Dense(512, activation='relu'))
model_b.add(Dropout(0.5))
model_b.add(Dense(classes, activation='softmax'))

model_b.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_b.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_8 (Conv3D)           (None, 30, 120, 120, 32)  2624      
                                                                 
 activation_8 (Activation)   (None, 30, 120, 120, 32)  0         
                                                                 
 conv3d_9 (Conv3D)           (None, 30, 120, 120, 32)  27680     
                                                                 
 activation_9 (Activation)   (None, 30, 120, 120, 32)  0         
                                                                 
 max_pooling3d_8 (MaxPooling  (None, 10, 40, 40, 32)   0         
 3D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 10, 40, 40, 32)    0         
                                                      

In [37]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [38]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 1.9094 - categorical_accuracy: 0.1976  Source path =  Project_data/val ; batch size = 10

Epoch 1: saving model to model_init_2023-02-2207_54_06.822453/model-00001-1.90942-0.19759-1.60840-0.23000.h5
67/67 [==============================] - 5012s 75s/step - loss: 1.9094 - categorical_accuracy: 0.1976 - val_loss: 1.6084 - val_categorical_accuracy: 0.2300 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 1.6107 - categorical_accuracy: 0.1841 
Epoch 2: saving model to model_init_2023-02-2207_54_06.822453/model-00002-1.61070-0.18408-1.60847-0.24000.h5
67/67 [==============================] - 1758s 26s/step - loss: 1.6107 - categorical_accuracy: 0.1841 - val_loss: 1.6085 - val_categorical_accuracy: 0.2400 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 1.6104 - categorical_accuracy: 0.1592 
Epoch 3: saving mode

Changing x,y,z values - 1

In [39]:
x = 30 # number of frames
y = 60 # image width
z = 60 # image height

In [40]:
# Define model b
model_b = Sequential()
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu')) 
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Flatten())
model_b.add(Dense(512, activation='relu'))
model_b.add(Dropout(0.5))
model_b.add(Dense(classes, activation='softmax'))

model_b.compile(optimizer=keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_b.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_12 (Conv3D)          (None, 30, 60, 60, 32)    2624      
                                                                 
 activation_12 (Activation)  (None, 30, 60, 60, 32)    0         
                                                                 
 conv3d_13 (Conv3D)          (None, 30, 60, 60, 32)    27680     
                                                                 
 activation_13 (Activation)  (None, 30, 60, 60, 32)    0         
                                                                 
 max_pooling3d_10 (MaxPoolin  (None, 10, 20, 20, 32)   0         
 g3D)                                                            
                                                                 
 dropout_7 (Dropout)         (None, 10, 20, 20, 32)    0         
                                                      

In [41]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [42]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 1.6403 - categorical_accuracy: 0.2293 Source path =  Project_data/val ; batch size = 10

Epoch 1: saving model to model_init_2023-02-2207_54_06.822453/model-00001-1.64033-0.22926-1.60086-0.29000.h5
67/67 [==============================] - 1478s 22s/step - loss: 1.6403 - categorical_accuracy: 0.2293 - val_loss: 1.6009 - val_categorical_accuracy: 0.2900 - lr: 0.0010
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 1.5521 - categorical_accuracy: 0.2687
Epoch 2: saving model to model_init_2023-02-2207_54_06.822453/model-00002-1.55209-0.26866-1.42552-0.31000.h5
67/67 [==============================] - 584s 9s/step - loss: 1.5521 - categorical_accuracy: 0.2687 - val_loss: 1.4255 - val_categorical_accuracy: 0.3100 - lr: 0.0010
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 1.4781 - categorical_accuracy: 0.2985
Epoch 3: saving model to 

Changing Batch size to 20

In [ ]:
train_generator = generator_ex(train_path, train_doc, 20)
val_generator = generator_ex(val_path, val_doc, 20)

In [ ]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Changing Batch size to 30

In [ ]:
train_generator = generator_ex(train_path, train_doc, 30)
val_generator = generator_ex(val_path, val_doc, 30)

In [ ]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Changing Batch size to 40 

In [ ]:
train_generator = generator_ex(train_path, train_doc, 40)
val_generator = generator_ex(val_path, val_doc, 40)

In [ ]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Change Optimizer to Adadelta

In [ ]:
model_b.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_b.summary()

In [ ]:
train_generator = generator_ex(train_path, train_doc, 40)
val_generator = generator_ex(val_path, val_doc, 40)

In [ ]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Change epoch to 20 

In [ ]:
num_epochs = 20 # choose the number of epochs
print ('epochs =', num_epochs)

In [ ]:
model_b.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_b.summary()

In [ ]:
train_generator = generator_ex(train_path, train_doc, 40)
val_generator = generator_ex(val_path, val_doc, 40)

In [ ]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Model C 

In [ ]:
model_c = Sequential()
model_c.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding="same"))
model_c.add(Activation('relu'))
model_c.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_c.add(Dropout(0.25))

model_c.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_c.add(Dropout(0.25))

model_c.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_c.add(Dropout(0.25))

model_c.add(Flatten())
model_c.add(Dense(512, activation='relu'))
model_c.add(BatchNormalization())
model_c.add(Dropout(0.5))
model_c.add(Dense(classes, activation='softmax'))
model_c.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_c.summary()

In [ ]:
# Changing back batch size, images per frame, height and width of image
batch_size = 10
x = 30 # number of frames
y = 120 # image width
z = 120# image height
num_epochs = 20

In [ ]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [ ]:
model_c.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Model D 

In [ ]:
input_shape=(x,y,z,channel)

nb_filters = [8,16,32,64]
nb_dense = [1000, 500, 5]
# Define model
model_d = Sequential()

model_d.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_d.add(BatchNormalization())
model_d.add(Activation('relu'))

model_d.add(MaxPooling3D(pool_size=(2,2,2)))

model_d.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_d.add(BatchNormalization())
model_d.add(Activation('relu'))

model_d.add(MaxPooling3D(pool_size=(2,2,2)))

model_d.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_d.add(BatchNormalization())
model_d.add(Activation('relu'))

model_d.add(MaxPooling3D(pool_size=(2,2,2)))

model_d.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_d.add(BatchNormalization())
model_d.add(Activation('relu'))

model_d.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_d.add(Flatten())

model_d.add(Dense(nb_dense[0], activation='relu'))
model_d.add(Dropout(0.5))

model_d.add(Dense(nb_dense[1], activation='relu'))
model_d.add(Dropout(0.5))

#softmax layer
model_d.add(Dense(nb_dense[2], activation='softmax'))
model_d.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_d.summary()


In [ ]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10
model_d.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Changing Model with images per frame, width and height of the images

In [ ]:
x = 30 # number of frames
y = 64 # image width
z = 64 # image height 

classes = 5
channel = 1

In [ ]:
model_e = Sequential()
model_e.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding="same"))
model_e.add(Activation('relu'))
model_e.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_e.add(Dropout(0.25))

model_e.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_e.add(Dropout(0.25))

model_e.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_e.add(Dropout(0.25))

model_e.add(Flatten())
model_e.add(Dense(512, activation='relu'))
model_e.add(BatchNormalization())
model_e.add(Dropout(0.5))
model_e.add(Dense(classes, activation='softmax'))
model_e.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_e.summary()

In [ ]:
batch_size = 5
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 20
model_e.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Model F

In [ ]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 1
nb_dense = [1000, 500, 5]

input_shape=(x,y,z,channel)

# Define model
model_f = Sequential()

model_f.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

model_f.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

model_f.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

model_f.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_f.add(Flatten())

model_f.add(Dense(nb_dense[0], activation='relu'))
model_f.add(Dropout(0.5))

model_f.add(Dense(nb_dense[1], activation='relu'))
model_f.add(Dropout(0.5))

#softmax layer
model_f.add(Dense(nb_dense[2], activation='softmax'))
model_f.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_f.summary()

In [ ]:
batch_size = 10
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 30
model_f.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Model G

In [ ]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 3

input_shape=(x,y,z,channel)

# Define model
model_g = Sequential()

model_g.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

model_g.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

model_g.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

model_g.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_g.add(Activation('relu'))
model_g.add(Dropout(0.25))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_g.add(Flatten())

model_g.add(Dense(nb_dense[0], activation='relu'))
model_g.add(Dropout(0.5))

model_g.add(Dense(nb_dense[1], activation='relu'))
model_g.add(Dropout(0.5))

#softmax layer
model_g.add(Dense(nb_dense[2], activation='softmax'))
model_g.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_g.summary()

In [ ]:
batch_size = 10
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10
model_g.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Final Model

In [ ]:
nb_filters = [8,16,32,64]
nb_dense = [256, 128, 5]

# Input
input_shape=(30,120,120,3)

# Define model
model_final = Sequential()

model_final.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_final.add(BatchNormalization())
model_final.add(Activation('relu'))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

model_final.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_final.add(BatchNormalization())
model_final.add(Activation('relu'))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

model_final.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_final.add(BatchNormalization())
model_final.add(Activation('relu'))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

model_final.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_final.add(Activation('relu'))
model_final.add(Dropout(0.25))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_final.add(Flatten())

model_final.add(Dense(nb_dense[0], activation='relu'))
model_final.add(Dropout(0.5))

model_final.add(Dense(nb_dense[1], activation='relu'))
model_final.add(Dropout(0.5))

#softmax layer
model_final.add(Dense(nb_dense[2], activation='softmax'))

In [ ]:
optimiser = keras.optimizers.Adam() #write your optimizer
model_final.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_final.summary())

In [ ]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [ ]:
batch_size = 10
num_epochs = 20
model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

In [ ]:
batch_size = 32
num_epochs = 20
model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

In [ ]:
batch_size = 64
num_epochs = 20

train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)


model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

In [ ]:
batch_size = 40
num_epochs = 20

train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)


model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)